In [363]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [364]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [365]:
df.shape

(569, 33)

In [366]:
df.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32'],
      dtype='object')

In [367]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [368]:
df[['diagnosis']].value_counts()
'''Malignant (cancerous) → 1
Benign (non-cancerous) → 0'''

'Malignant (cancerous) → 1\nBenign (non-cancerous) → 0'

In [369]:
# Display original column names
print("Original Columns:\n", df.columns)

# Sort column names
sorted_columns = sorted(df.columns)

# Reorder DataFrame columns based on sorted names
df = df[sorted_columns]

# Display sorted column names
print("Sorted Columns:\n", df.columns)

Original Columns:
 Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32'],
      dtype='object')
Sorted Columns:
 Index(['Unnamed: 32', 'area_mean', 'area_se', 'area_worst', 'compactness_mean',
       'compactness_se', 'compactness_worst', 'concave points_mean',
       'concave points_se', 'concave points_worst', 'concavity_mean',
       'concavity_se', 'concavity_worst', 'diagnosis',
       'fractal_dimension

In [370]:
df.shape

(569, 33)

In [371]:
#cleaning the dataset
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

<ipython-input-371-320d0380df3c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['id', 'Unnamed: 32'], inplace= True)


In [372]:
def missing_data(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
#    return tt
    return(np.transpose(tt))

In [373]:
missing_data(df)

,area_mean,area_se,area_worst,compactness_mean,compactness_se,compactness_worst,concave points_mean,concave points_se,concave points_worst,concavity_mean,...,radius_worst,smoothness_mean,smoothness_se,smoothness_worst,symmetry_mean,symmetry_se,symmetry_worst,texture_mean,texture_se,texture_worst
Total,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Percent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Types,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,...,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64


In [374]:
df.head()

,area_mean,area_se,area_worst,compactness_mean,compactness_se,compactness_worst,concave points_mean,concave points_se,concave points_worst,concavity_mean,...,radius_worst,smoothness_mean,smoothness_se,smoothness_worst,symmetry_mean,symmetry_se,symmetry_worst,texture_mean,texture_se,texture_worst
0,1001.0,153.40,2019.0,0.27760,0.04904,0.6656,0.14710,0.01587,0.2654,0.3001,...,25.38,0.11840,0.006399,0.1622,0.2419,0.03003,0.4601,10.38,0.9053,17.33
1,1326.0,74.08,1956.0,0.07864,0.01308,0.1866,0.07017,0.01340,0.1860,0.0869,...,24.99,0.08474,0.005225,0.1238,0.1812,0.01389,0.2750,17.77,0.7339,23.41
2,1203.0,94.03,1709.0,0.15990,0.04006,0.4245,0.12790,0.02058,0.2430,0.1974,...,23.57,0.10960,0.006150,0.1444,0.2069,0.02250,0.3613,21.25,0.7869,25.53
3,386.1,27.23,567.7,0.28390,0.07458,0.8663,0.10520,0.01867,0.2575,0.2414,...,14.91,0.14250,0.009110,0.2098,0.2597,0.05963,0.6638,20.38,1.1560,26.50
4,1297.0,94.44,1575.0,0.13280,0.02461,0.2050,0.10430,0.01885,0.1625,0.1980,...,22.54,0.10030,0.011490,0.1374,0.1809,0.01756,0.2364,14.34,0.7813,16.67


In [375]:
df.columns

Index(['area_mean', 'area_se', 'area_worst', 'compactness_mean',
       'compactness_se', 'compactness_worst', 'concave points_mean',
       'concave points_se', 'concave points_worst', 'concavity_mean',
       'concavity_se', 'concavity_worst', 'diagnosis',
       'fractal_dimension_mean', 'fractal_dimension_se',
       'fractal_dimension_worst', 'perimeter_mean', 'perimeter_se',
       'perimeter_worst', 'radius_mean', 'radius_se', 'radius_worst',
       'smoothness_mean', 'smoothness_se', 'smoothness_worst', 'symmetry_mean',
       'symmetry_se', 'symmetry_worst', 'texture_mean', 'texture_se',
       'texture_worst'],
      dtype='object')

In [376]:
X = df.drop(columns=['diagnosis']).values  # Do not use inplace=True
y = df[['diagnosis']].values

In [377]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [378]:
#df['diagnosis'].value_counts()
#print(X_train.dtypes)


In [379]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Label Encoding

In [380]:
y_train.shape

(455, 1)

In [381]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [382]:
y_test.shape

(114,)

In [383]:
X_train.shape

(455, 30)

In [384]:
import torch
print(torch.__version__)

2.5.1+cu124


In [385]:
if torch.cuda.is_available():
    print("GPU is available!")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("GPU not available. Using CPU.")

GPU is available!
Using GPU: Tesla T4


In [386]:
n_tr    = X_train.shape[0]
print(n_tr)

n_ts    = X_test.shape[0]
print(n_ts)

n_feat  = X_train.shape[1]
print(n_feat)

n_class = (np.unique(y_test).size)
print(n_class)

455
114
30
2


In [387]:
# now print those details
print("Num training= %d" % n_tr)
print("Num test=     %d" % n_ts)
print("Num features= %d" % n_feat)
print("Num classes=  %d" % n_class)

Num training= 455
Num test=     114
Num features= 30
Num classes=  2


In [388]:
import torch
import torch.nn as nn
import torch.optim as optim

In [389]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)  # Add dimension
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

In [390]:
X_train_tensor.shape

torch.Size([455, 30])

In [391]:
y_train_tensor.shape

torch.Size([455, 1])

In [392]:
# Network Parameters
input_size = X_train_tensor.shape[1]  # Number of input features (30)
hidden_size1 = 256
hidden_size2 = 128
output_size = 1    # Binary classification (1 output)
learning_rate = 0.05
epochs = 50

In [393]:
weights = {
    "w1": torch.randn(hidden_size1, input_size, requires_grad=False) * 0.01,
    "b1": torch.zeros(hidden_size1, requires_grad=False),
    "w2": torch.randn(hidden_size2, hidden_size1, requires_grad=False) * 0.01,
    "b2": torch.zeros(hidden_size2, requires_grad=False),
    "w3": torch.randn(output_size, hidden_size2, requires_grad=False) * 0.01,
    "b3": torch.zeros(output_size, requires_grad=False),
}

In [394]:
print(X_train_tensor.shape)
print(weights['w1'].shape)

torch.Size([455, 30])
torch.Size([256, 30])


In [395]:
# Activation Functions
def relu(x):
    return torch.maximum(torch.tensor(0.0), x)

def relu_derivative(x):
    return (x > 0).float()

def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

# Binary Cross-Entropy Loss
def binary_cross_entropy(y, y_hat):
    eps = 1e-12  # To prevent log(0)
    loss = -torch.mean(y * torch.log(y_hat + eps) + (1 - y) * torch.log(1 - y_hat + eps))
    return loss


Training

In [396]:
for epoch in range(epochs):
    # Forward Pass
    z1 = torch.matmul(X_train_tensor, weights["w1"].T) + weights["b1"]
    a1 = relu(z1)
    z2 = torch.matmul(a1, weights["w2"].T) + weights["b2"]
    a2 = relu(z2)
    z3 = torch.matmul(a2, weights["w3"].T) + weights["b3"]
    y_hat = sigmoid(z3)  #for binary classification

    loss = binary_cross_entropy(y_train_tensor, y_hat)


    # Backward Pass (Gradients Calculation)
    dz3 = y_hat - y_train_tensor  # Output layer gradient
    dw3 = torch.matmul(dz3.T, a2) / X_train_tensor.shape[0]  # Gradient w.r.t. weights (w3)
    db3 = torch.sum(dz3, dim=0) / X_train_tensor.shape[0]  # Gradient w.r.t. bias (b3)

    dz2 = torch.matmul(dz3, weights["w3"]) * relu_derivative(z2)  # Hidden layer 2 gradient
    dw2 = torch.matmul(dz2.T, a1) / X_train_tensor.shape[0]  # Gradient w.r.t. weights (w2)
    db2 = torch.sum(dz2, dim=0) / X_train_tensor.shape[0]  # Gradient w.r.t. bias (b2)

    dz1 = torch.matmul(dz2, weights["w2"]) * relu_derivative(z1)  # Hidden layer 1 gradient
    dw1 = torch.matmul(dz1.T, X_train_tensor) / X_train_tensor.shape[0]  # Gradient w.r.t. weights (w1)
    db1 = torch.sum(dz1, dim=0) / X_train_tensor.shape[0]  # Gradient w.r.t. bias (b1)

    # Updating Weights and Biases
    weights["w3"] -= learning_rate * dw3
    weights["b3"] -= learning_rate * db3
    weights["w2"] -= learning_rate * dw2
    weights["b2"] -= learning_rate * db2
    weights["w1"] -= learning_rate * dw1
    weights["b1"] -= learning_rate * db1


    # Print loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}")

Epoch 10/50, Loss: 0.6874
Epoch 20/50, Loss: 0.6823
Epoch 30/50, Loss: 0.6784
Epoch 40/50, Loss: 0.6753
Epoch 50/50, Loss: 0.6728


**Testing**

In [397]:
z1_test = torch.matmul(X_test_tensor, weights["w1"].T) + weights["b1"]
a1_test = relu(z1_test)
z2_test = torch.matmul(a1_test, weights["w2"].T) + weights["b2"]
a2_test = relu(z2_test)
z3_test = torch.matmul(a2_test, weights["w3"].T) + weights["b3"]
y_test_hat = sigmoid(z3_test)

In [398]:
# Calculate test loss
test_loss = binary_cross_entropy(y_test_tensor, y_test_hat)

In [399]:
y_test_pred = (y_test_hat > 0.5).float()
accuracy = (y_test_pred == y_test_tensor).float().mean().item()
# Convert probabilities to binary predictions
print(f"Test Loss: {test_loss.item():.4f}")
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Loss: 0.6618
Test Accuracy: 66.67%
